In [4]:
import cv2
import numpy as np
import math
import pandas as pd
from matplotlib import pyplot as plt
from img_listing import image_list
from skimage.feature import greycomatrix, greycoprops
from hist_3d import hist_3d2
from scipy.misc import imresize

In [5]:

counter=0
filename = 'C:\WORK\Engineering Physics\Tugas Terakhir\Project\code\combine_img'
img_list = image_list(filename)
np.savetxt("img_list.txt", img_list, delimiter=" ", newline = "\n", fmt="%s")

In [6]:
img_list

['ISIC_0000000.jpg',
 'ISIC_0000001.jpg',
 'ISIC_0000002.jpg',
 'ISIC_0000003.jpg',
 'ISIC_0000005.jpg',
 'ISIC_0000006.jpg',
 'ISIC_0000007.jpg',
 'ISIC_0000008.jpg',
 'ISIC_0000009.jpg',
 'ISIC_0000010.jpg',
 'ISIC_0000011.jpg',
 'ISIC_0000012.jpg',
 'ISIC_0000013.jpg',
 'ISIC_0000014.jpg',
 'ISIC_0000015.jpg',
 'ISIC_0000016.jpg',
 'ISIC_0000017.jpg',
 'ISIC_0000018.jpg',
 'ISIC_0000019.jpg',
 'ISIC_0000020.jpg',
 'ISIC_0000021.jpg',
 'ISIC_0000022.jpg',
 'ISIC_0000023.jpg',
 'ISIC_0000024.jpg',
 'ISIC_0000025.jpg',
 'ISIC_0000026.jpg',
 'ISIC_0000027.jpg',
 'ISIC_0000028.jpg',
 'ISIC_0000029.jpg',
 'ISIC_0000030.jpg',
 'ISIC_0000031.jpg',
 'ISIC_0000032.jpg',
 'ISIC_0000033.jpg',
 'ISIC_0000034.jpg',
 'ISIC_0000035.jpg',
 'ISIC_0000037.jpg',
 'ISIC_0000038.jpg',
 'ISIC_0000039.jpg',
 'ISIC_0000040.jpg',
 'ISIC_0000041.jpg',
 'ISIC_0000042.jpg',
 'ISIC_0000043.jpg',
 'ISIC_0000044.jpg',
 'ISIC_0000045.jpg',
 'ISIC_0000046.jpg',
 'ISIC_0000047.jpg',
 'ISIC_0000048.jpg',
 'ISIC_000004

In [ ]:
num8bit = np.linspace(0,255,256,dtype=int)
a=[]
c=[]
for i in range(256):
    R=('R'+str(num8bit[i]))
    for j in range(256):
        G=('G'+str(num8bit[j]))
        a = (R+G)
        for k in range(256):
            B=('B'+str(num8bit[k]))
            a=(R+G+B)
            c.append(a)

In [ ]:
isic_data = pd.read_csv('combine_img' + '\metadata1.csv')
isic_data.columns = isic_data.columns.str.replace('.','_')


In [ ]:
df.head()

In [ ]:
a = 0
a

In [ ]:
a = a+1
a

In [ ]:
combine_hist.shape[0]/(255**3)

In [ ]:
flat_hist.shape

In [ ]:
combine_hist = np.zeros([(64**3)+1,len(img_list)])
combine_hist.shape

In [ ]:

j=0
for i in img_list :
    #CHOOSE IMAGE
    
    print('IMAGE : ' + i)
    name = i.split('.jpg')[0]

    img = cv2.imread(filename + '/' + i)
    imy,imx,imz = np.shape(img)
    shortening = 5
    img = img[shortening:imy-shortening,shortening:imx-shortening,:]
    imy,imx,imz = np.shape(img)
    gs = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    hsv=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    
    
    ret1,th1 = cv2.threshold(gs,25,255,cv2.THRESH_BINARY)
    
    ckernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8))
    closing = cv2.morphologyEx(th1, cv2.MORPH_CLOSE, ckernel,iterations = 4)
    
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    erosion = cv2.erode(closing,kernel,iterations = 50)
    
    derm_scope = erosion
     
     
     
    #REMOVING HAIR FILTERING------------- START
     
    #gs_filter = cv2.medianBlur(gs,21) #median blur
    #gs_filter = cv2.GaussianBlur(gs,(31,31),0) #Gaussian Blur
    #gs_filter = cv2.blur(gs,(31,31)) #Averaging
    #REMOVING HAIR FILTERING------------- END
     
     
     
    ret,th = cv2.threshold(gs,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
     
    #REMOVING HAIR MORPH TRANSFORM------------- START
    #Morphologic Transform ------------- START
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    erosion = cv2.erode(th,kernel,iterations = int(min([imx,imy])/100))
    dilation = cv2.dilate(erosion,kernel,iterations = int(min([imx,imy])/100))
    #Morphologic Transform ------------- End
    #REMOVING HAIR MORPH TRANSFORM------------- END
     
    segment_mask = dilation
     
     
     
     
    nmask =  cv2.bitwise_and(derm_scope, segment_mask)
     
    contours,hierarchy = cv2.findContours(nmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
     
    contimg = img.copy()
     
    height, width, channels = img.shape
    all_cnt_bimg = np.zeros((height,width,1), np.uint8)
    cv2.drawContours(all_cnt_bimg, contours, -1, 255, 1)

    
    
    
    if len(contours) != 0:
        
        diagnosis = isic_data[isic_data.name == i.split('.')[0]]['meta_clinical_diagnosis']
        diagnosis_n = []
        if diagnosis.iloc[0] == 'melanoma':
            diagnosis_n = 1
        elif diagnosis.iloc[0] == 'nevus':
            diagnosis_n = 0
            
        
        #find the biggest area
        c = max(contours, key = cv2.contourArea)
    
        # draw in blue the contours that were founded
        cv2.drawContours(contimg, c, -1, 255, 8)
    
        x,y,w,h = cv2.boundingRect(c)
        # draw the book contour (in green)
        cv2.rectangle(contimg,(x,y),(x+w,y+h),(0,255,0),8)
    
        #Finding center of mass
        M = cv2.moments(c)
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
        t_cx = cx-(imx/2)
        t_cy = cy-(imy/2)
        
    
        #CREATING BORDER 
        square_r = np.maximum(imx,imy)
        bimg = np.zeros((height,width,1), np.uint8)
        cv2.drawContours(bimg, c, -1, 255, 5)
        flood_img = bimg.copy()
        maskfill = np.zeros((height+2, width+2), np.uint8)    
        cv2.floodFill(flood_img, maskfill, (cx,cy), 255)
        segment = cv2.bitwise_and(img, img, mask=flood_img)


    #-------------------------------------------------------------- COLOR START

    # INVERSE SEGMENTATION OF SKIN


    inv_flood_img = cv2.bitwise_not(flood_img)
    inv_segment = cv2.bitwise_and(img, img, mask=inv_flood_img)
#        
#    inv_segment1 = cv2.cvtColor(inv_segment, cv2.COLOR_BGR2HSV)
#    segment1 = cv2.cvtColor(segment, cv2.COLOR_BGR2HSV)

    inv_segment1 = inv_segment
    segment1 = segment
    
    # SPLITTING RGB CHANNEL
    s_R = inv_segment1[:,:,2]
    s_G = inv_segment1[:,:,1]
    s_B = inv_segment1[:,:,0]

    # FIND SKIN AVERAGE VALUE
    ave_s_R = np.sum(s_R)/(np.sum(inv_flood_img)/255)
    ave_s_G = np.sum(s_G)/(np.sum(inv_flood_img)/255)
    ave_s_B = np.sum(s_B)/(np.sum(inv_flood_img)/255)

#    print('red skin average val :' + str(ave_s_R))
#    print('Green skin average val :' + str(ave_s_G))
#    print('Blue skin average val :' + str(ave_s_B))
#    
    # CONVERT RGB DATA TYPE
    R_mat = segment1[:,:,2].astype('int32')
    G_mat = segment1[:,:,1].astype('int32')
    B_mat = segment1[:,:,0].astype('int32')


    col_segment = np.zeros([imy,imx,3])
    col_segment[:,:,2] = R_mat
    col_segment[:,:,1] = G_mat
    col_segment[:,:,0] = B_mat
    
    col_segment_point = col_segment.reshape([(imx*imy),3])
    
    points = col_segment_point
    
    hist,binedges = hist_3d2(col_segment_point,64)

    bg_px = np.sum(inv_flood_img)/255
    
    a = np.array([255-ave_s_B,255-ave_s_G,255-ave_s_R])
    b = np.uint(np.floor(a*16/(256+255)))
    
    hist[b[0],b[1],b[2]]=hist[b[0],b[1],b[2]] - bg_px
    
    norm_hist = hist/np.sum(hist)

    #-------------------------------------------------------------- COLOR END
    
    flat_hist = norm_hist.flatten()
    flat_hist = np.append(flat_hist, diagnosis_n)
    combine_hist[:,j] = flat_hist
    j = j+1
    
    
    
#     cv2.imwrite('contimage/contour' + str(i), contimg)


#     df2 = pd.DataFrame(data = {'Diagnosis' : [diagnosis_n],
#                                'Normalize_Hist': [hist]}, index = [name])
#     print(str(df2))
#     df = df.append(df2)
    
#     print('\n')
#     print('\n')
#     print('\n')
    
#     counter=counter + 1
#     if counter >= 500:
#         df.to_csv('Feature Data.csv')
#         df.to_pickle('Feature Data.pk')    
#         counter = 0
#     counter = counter + 1

In [ ]:
combine_hist[0].shape

In [ ]:
full_data = np.append(combine_hist,img_list)